In [ ]:
import numpy as np
import pandas as pd
from scipy import ndimage

import os
import gc

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM, z_at_value

from field_util import Field, CartCoord
from FGPA_to_skewers import Mock_skewer

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
np.random.seed(114514)

In [ ]:
data_path = '../Data/'
plot_path = '../Plot/'

In [ ]:
cosmo = FlatLambdaCDM(H0=100* u.km / u.s / u.Mpc, Om0=0.315) # Metin original
mycoord = CartCoord(150.14205192829834, 2.224237689411875, cosmo) # Used in Cosmic Birth

In [ ]:
# extract metadata

# CLAMATO galaxies
clamato_gal = '../Data/CLAMATO/list_tomo_input_2020_mod.txt'
clm_gal_list = pd.read_csv(clamato_gal, sep = '\t', header = 0)

# CLAMATO full catalog
clamato_full = '../Data/CLAMATO/cl2020_valueadded_release_20200602_mod.txt'
clm_full_list = pd.read_csv(clamato_full)

clm_gal_with_SN = pd.merge(clm_gal_list, clm_full_list, left_on='TomoID', right_on='TOMO_ID')
# - S/N_1: Estimated Lya-forest S/N at 2.05<z<2.15, -9.0 denotes no estimate
# - S/N_2: Estimated Lya-forest S/N at 2.15<z<2.35, -9.0 denotes no estimate
# - S/N_2: Estimated Lya-forest S/N at 2.35<z<2.55, -9.0 denotes no estimate
clm_gal_short = clm_gal_with_SN[['TomoID', 'S/N_1', 'S/N_2', 'S/N_3']]

In [ ]:
# CLAMATO pixel
clamato_pixel = data_path+'CLAMATO/pixel_data_dr2.bin'
x_0, y_0, z_0 = mycoord.orig_to_box(150.14205192829834, 2.224237689411875, 2.05)
with open(clamato_pixel,'r') as f:
    npix = np.fromfile(f, dtype=np.int32, count=1)[0]
    f.seek(4)
    pixel_data = np.fromfile(f,dtype=np.float64).reshape(npix,5)

x_new, y_new, z_new = mycoord.orig_to_box(clm_gal_list['ra'], clm_gal_list['dec'], 2.30)
coord_new = np.zeros([pixel_data.shape[0], 3])
coord_new[:, 0] = pixel_data[:, 2] + x_0.value
for i, gal in enumerate(clm_gal_list.iloc):
    coord_new[int(gal['i_start']):int(gal['i_end'])+1,1] = y_new[i].value
    coord_new[int(gal['i_start']):int(gal['i_end'])+1,2] = z_new[i].value

tomo_id = np.zeros([pixel_data.shape[0], 1], dtype=int)
for i, gal in enumerate(clm_gal_list.iloc):
    tomo_id[int(gal['i_start']):int(gal['i_end'])+1, 0] = gal['TomoID']

pixel_data = np.hstack([tomo_id, pixel_data, coord_new])
pixel_data_df = pd.DataFrame(pixel_data, columns = ['tomo_id', 'x', 'y', 'z', 'sigma_f', 'delta_f', 
                                                    'x_new', 'y_new', 'z_new'])
pixel_data_df['tomo_id'] = (pixel_data_df['tomo_id']).astype(int)
N_pix = len(pixel_data_df)

In [ ]:
# merge SNR data from CLAMATO
clm_red_list = clm_full_list[['TOMO_ID', 'ZSPEC']]
pixel_data_df = pd.merge(pixel_data_df, clm_red_list, left_on = 'tomo_id', right_on = 'TOMO_ID')
del pixel_data_df['TOMO_ID']

def z_from_dist(distance, cosmo):
    dummyred = np.linspace(0.,10.,10000)
    dummydist = cosmo.comoving_distance(dummyred)
    res = np.interp(distance,dummydist,dummyred)
    return (res)
pixel_data_df['red'] = z_from_dist(pixel_data_df['x_new']*u.Mpc, cosmo)

pixel_data_temp = pd.merge(pixel_data_df, clm_gal_short, left_on='tomo_id', right_on='TomoID')
SNR = ((pixel_data_temp['red'] < 2.15) * (pixel_data_temp['red'] > 2.05) * pixel_data_temp['S/N_1'] +
       (pixel_data_temp['red'] < 2.35) * (pixel_data_temp['red'] > 2.15) * pixel_data_temp['S/N_2'] +
       (pixel_data_temp['red'] < 2.55) * (pixel_data_temp['red'] > 2.35) * pixel_data_temp['S/N_3'])
SNR[SNR<=0] = np.nan
pixel_data_temp['S/N'] = SNR
pixel_data_temp = pixel_data_temp.dropna()

In [ ]:
# load tau file
tau_list = [fn for fn in os.listdir(data_path+'lores_tau/') if fn[-7:]=='tau.npy']
fn = tau_list[0]

# from this part we will make a loop
# load tau map

In [ ]:
# generate mock skewers as dachshund input
np.random.seed(1919)

for fn in tau_list:
    for j in range(20):
        fn_new = '_'.join(fn.split('_')[1:3])
        tau_data = np.load(data_path+'lores_tau/'+fn)[:,::-1,:]
        # convert to transmission
        tran_data = np.exp(-tau_data)
        delta_F = np.exp(-tau_data) / np.exp(-tau_data).mean() - 1
        field_trans =  Field(3550.*u.Mpc, -50.*u.Mpc, -50.*u.Mpc, 2*u.Mpc, tran_data)
        _ = field_trans.smooth(2*u.Mpc)
        # raw data w/o noise
        res = field_trans.field_eval(np.array(pixel_data_temp['x_new'])*u.Mpc, 
                                     np.array(pixel_data_temp['y_new'])*u.Mpc,
                                     np.array(pixel_data_temp['z_new'])*u.Mpc)
        pixel_data_temp['F_raw'] = res

        # turbulate transmission with a gaussian
        # pixel_data_temp['F_obs'] = pixel_data_temp['F_raw']
        pixel_data_temp['delta_F_sim'] = pixel_data_temp['F_raw'] / pixel_data_temp['F_raw'].mean() - 1
        pixel_data_temp['sigma_F_sim'] = (1. / pixel_data_temp['S/N'] + 0.1) / pixel_data_temp['F_raw'].mean()

        # add noise based on SNR
        pixel_data_temp['delta_F_rec'] = pixel_data_temp['delta_F_sim'] + np.random.normal(scale=pixel_data_temp['sigma_F_sim'])

        # export the result
        pix_sim = np.array(pixel_data_temp[['x', 'y', 'z', 'sigma_F_sim', 'delta_F_rec']])
        pix_sim[:,3][np.isnan(pix_sim[:,3])] = 10 # large var
        pix_sim[:,4][np.isnan(pix_sim[:,4])] = 0 # null value

        pixel_data_temp = pixel_data_temp.dropna() # some pixels go beyond the red range
        N_pix = len(pixel_data_temp)

        # pix_sim.tofile(data_path+"/mock_skewers_ex/pixel_data_{}.bin".format(fn[:-14]))
        pix_sim.tofile(data_path+"/mock_skewers_ex/pixel_data_{}_{}_true_SNR.bin".format(fn_new, j))
        pix_sim.tofile(data_path+"/mock_skewers_hires/pixel_data_{}_{}_true_SNR.bin".format(fn_new, j))